In [ ]:
# Importações para deep learning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Dense, LSTM, GRU, SimpleRNN, Embedding, 
                                   Conv1D, MaxPooling1D, GlobalMaxPooling1D,
                                   Dropout, BatchNormalization, Attention,
                                   Input, Bidirectional, TimeDistributed)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Scikit-learn para comparação
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix

# Visualização
import plotly.express as px
import plotly.graph_objects as go

# NLP
import nltk
from nltk.tokenize import word_tokenize

# Datasets
import sys
sys.path.append('..')
from datasets.textos_exemplo import *
from utils.nlp_utils import *

# Configurar TensorFlow
tf.random.set_seed(42)
print(f"🧠 TensorFlow {tf.__version__} carregado!")
print(f"GPU disponível: {tf.config.list_physical_devices('GPU')}")
print("🧠 Deep Learning para NLP pronto!")


In [ ]:
# Preparar dados para deep learning
reviews_positivos = [
    "Excelente produto, recomendo muito!",
    "Adorei a qualidade, superou minhas expectativas.",
    "Muito bom, chegou rápido e bem embalado.",
    "Produto incrível, vale cada centavo!",
    "Estou muito satisfeito com a compra.",
    "Qualidade excepcional, comprarei novamente.",
    "Fantástico! Melhor que imaginava.",
    "Produto maravilhoso, entrega rápida.",
    "Simplesmente perfeito, recomendo a todos.",
    "Excelente custo-benefício, muito bom produto.",
    "Adorei o produto, muito útil no dia a dia.",
    "Superou todas as expectativas, incrível!",
    "Produto de alta qualidade, vale a pena.",
    "Muito satisfeito, chegou antes do prazo.",
    "Excelente atendimento e produto top!"
]

reviews_negativos = [
    "Produto terrível, não recomendo.",
    "Muito ruim, dinheiro jogado fora.",
    "Qualidade péssima, chegou quebrado.",
    "Horrível, não serve para nada.",
    "Decepcionante, esperava muito mais.",
    "Produto de baixa qualidade, muito caro.",
    "Não gostei, vou devolver.",
    "Péssimo atendimento e produto ruim.",
    "Não vale o preço, muito caro para pouca qualidade.",
    "Produto defeituoso, não funcionou.",
    "Muito ruim, não recomendo a ninguém.",
    "Péssima qualidade, chegou danificado.",
    "Produto horrível, perda de tempo.",
    "Não gostei nada, muito decepcionante.",
    "Qualidade inferior, não vale o dinheiro."
]

# Criar dataset
texts = reviews_positivos + reviews_negativos
labels = [1] * len(reviews_positivos) + [0] * len(reviews_negativos)

print(f"📊 Dataset para Deep Learning:")
print(f"Total de textos: {len(texts)}")
print(f"Positivos: {sum(labels)}")
print(f"Negativos: {len(labels) - sum(labels)}")

# Configurações
MAX_WORDS = 1000
MAX_LEN = 50
EMBEDDING_DIM = 100

# Tokenização e preparação dos dados
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

# Converter textos para sequências
sequences = tokenizer.texts_to_sequences(texts)

# Padding das sequências
X = pad_sequences(sequences, maxlen=MAX_LEN, padding='post', truncating='post')
y = np.array(labels)

print(f"\n🔢 Dados processados:")
print(f"Forma de X: {X.shape}")
print(f"Forma de y: {y.shape}")
print(f"Vocabulário: {len(tokenizer.word_index)} palavras")

# Exemplo de sequência processada
print(f"\n📝 Exemplo de processamento:")
print(f"Texto original: '{texts[0]}'")
print(f"Sequência: {sequences[0]}")
print(f"Padded: {X[0]}")

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n🔀 Divisão dos dados:")
print(f"Treino: {X_train.shape}")
print(f"Teste: {X_test.shape}")

# Função para criar modelo LSTM
def create_lstm_model(vocab_size, embedding_dim, max_length, lstm_units=64):
    """Cria modelo LSTM para classificação binária"""
    
    model = Sequential([
        # Camada de embedding
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        
        # Dropout para regularização
        Dropout(0.3),
        
        # Camada LSTM
        LSTM(lstm_units, dropout=0.3, recurrent_dropout=0.3),
        
        # Camada densa
        Dense(32, activation='relu'),
        Dropout(0.5),
        
        # Camada de saída
        Dense(1, activation='sigmoid')
    ])
    
    return model

# Criar modelo
model = create_lstm_model(
    vocab_size=MAX_WORDS,
    embedding_dim=EMBEDDING_DIM,
    max_length=MAX_LEN,
    lstm_units=64
)

# Compilar modelo
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Mostrar arquitetura
print(f"\n🏗 Arquitetura do modelo:")
model.summary()

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=0.0001
)

# Treinar modelo
print(f"\n🚀 Treinando modelo LSTM...")
history = model.fit(
    X_train, y_train,
    batch_size=8,
    epochs=20,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Avaliar modelo
print(f"\n📊 Avaliação do modelo:")
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Acurácia no teste: {test_accuracy:.4f}")
print(f"Loss no teste: {test_loss:.4f}")

# Predições
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Relatório de classificação
print(f"\n📋 Relatório de classificação:")
print(classification_report(y_test, y_pred, target_names=['Negativo', 'Positivo']))

# Visualizar histórico de treinamento
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
ax1.plot(history.history['accuracy'], label='Treino')
ax1.plot(history.history['val_accuracy'], label='Validação')
ax1.set_title('Acurácia do Modelo')
ax1.set_xlabel('Época')
ax1.set_ylabel('Acurácia')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Loss
ax2.plot(history.history['loss'], label='Treino')
ax2.plot(history.history['val_loss'], label='Validação')
ax2.set_title('Loss do Modelo')
ax2.set_xlabel('Época')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Testar com novos textos
novos_textos = [
    "Este produto é fantástico, recomendo!",
    "Muito ruim, não gostei nada.",
    "Qualidade média, poderia ser melhor.",
    "Excelente compra, muito satisfeito!"
]

print(f"\n🧪 Testando com novos textos:")
for texto in novos_textos:
    # Processar texto
    seq = tokenizer.texts_to_sequences([texto])
    padded = pad_sequences(seq, maxlen=MAX_LEN, padding='post')
    
    # Predizer
    pred_proba = model.predict(padded, verbose=0)[0][0]
    pred_label = "Positivo" if pred_proba > 0.5 else "Negativo"
    
    print(f"'{texto}'")
    print(f"  → {pred_label} (confiança: {pred_proba:.3f})")
    print()
